In [19]:
import os
SAVE_PATH = os.getcwd()
os.makedirs(SAVE_PATH, exist_ok=True)

In [20]:
# 3. 시트 이름 확인
import pandas as pd
xls = pd.ExcelFile('20-24Year_PersonalStats_PAOver50pct.xlsx')
sheet_names = xls.sheet_names
print(" 포함된 시트 목록:", sheet_names)

 포함된 시트 목록: ['concated']


In [21]:
# 모든 시트를 하나의 DataFrame으로 통합
df_all = pd.concat([xls.parse(sheet_name=sheet) for sheet in sheet_names], ignore_index=True)

# 컬럼 소문자로 정리
df_all.columns = [col.strip().lower() for col in df_all.columns]

# 주요 컬럼만 추출
expected_cols = ['year', 'team', 'name', 'pa', 'ops', 'avg', 'obp', 'slg', 'rbi', 'sb', 'hr']
missing = [col for col in expected_cols if col not in df_all.columns]
print(" 누락 컬럼:", missing)

# 결측치 제거
df_clean = df_all[expected_cols].dropna()
print(" 정제된 데이터 수:", df_clean.shape)
df_clean.head()


 누락 컬럼: []
 정제된 데이터 수: (503, 11)


,year,team,name,pa,ops,avg,obp,slg,rbi,sb,hr
0,24,KIA,김도영,625,1.067,0.347,0.420,0.647,109,40,38
1,22,KIA,나성범,649,0.910,0.320,0.402,0.508,97,6,21
2,20,KIA,최형우,600,1.023,0.354,0.433,0.590,115,0,28
3,20,KIA,터커,631,0.955,0.306,0.398,0.557,113,0,32
4,22,KIA,소크라테스,554,0.848,0.311,0.354,0.494,77,12,17


In [22]:
df_filtered = df_clean[df_clean['pa'] > 100].copy()

# 5. 필요한 컬럼만 추출 및 정리
key_columns = ['Year', 'Team', 'Name', 'PA', 'OPS', 'AVG', 'OBP', 'SLG', 'RBI', 'SB', 'HR']
# 1. 컬럼명 모두 소문자로 정리
df_all.columns = [col.strip().lower() for col in df_all.columns]

# 2. 추출할 컬럼명도 소문자로 지정
key_columns = ['year', 'team', 'name', 'pa', 'ops', 'avg', 'obp', 'slg', 'rbi', 'sb', 'hr']

# 3. 필요한 컬럼만 추출
df_clean = df_all[key_columns].dropna()

# 4. 확인
print(" 정제된 데이터 크기:", df_clean.shape)
df_clean.head()

 정제된 데이터 크기: (503, 11)


,year,team,name,pa,ops,avg,obp,slg,rbi,sb,hr
0,24,KIA,김도영,625,1.067,0.347,0.420,0.647,109,40,38
1,22,KIA,나성범,649,0.910,0.320,0.402,0.508,97,6,21
2,20,KIA,최형우,600,1.023,0.354,0.433,0.590,115,0,28
3,20,KIA,터커,631,0.955,0.306,0.398,0.557,113,0,32
4,22,KIA,소크라테스,554,0.848,0.311,0.354,0.494,77,12,17


In [23]:
import pandas as pd

# 컬럼명 통일
df_all.columns = [col.strip().lower() for col in df_all.columns]

# 주요 컬럼만 필터링 (수동 매핑 필요할 수 있음)
expected_cols = ['year', 'team', 'name', 'pa', 'ops', 'avg', 'obp', 'slg', 'rbi', 'sb', 'hr']
missing_cols = [col for col in expected_cols if col not in df_all.columns]
print(" 누락 컬럼:", missing_cols)  # 없으면 바로 아래 실행

df_clean = df_all[expected_cols].dropna()
print("df_clean shape:", df_clean.shape)


 누락 컬럼: []
df_clean shape: (503, 11)


In [24]:
# 팀/연도별 선수 수 (조건 없이 전체)
team_year_counts = df_clean.groupby(['year', 'team']).size().sort_values()

# 전체 총 선수 수
total_players = len(df_clean)

# 출력
print(" 팀/연도별 선수 수:")
print(team_year_counts)

print(f"\n 전체 선수 수 (조건 없음): {total_players}")

#여기서 고질적인 문제가 하나 생긴다. 최적의 타순을 구하기 위해서 우린 우선적으로 필터링을 두 차례로 거쳐서 할 예정이었다.
1#1. 년도별 2. 팀별
#허나 데이터를 정리해보면 알 수 있지만 야구의 타순을 알기 위해서 필요한 최소한의 타자는 9명, 보면 알겠지만 대 부분의 타자들은 겨우 9명인 경우가 대 다수다.
#즉, 임의로 테스트를 할 때 유의미한 결과를 얻기 위한 데이터의 양 자체가 부실할 수 있다는 판단이다.

#따라서 이 머신 러닝에선 총 두 가지로 나눠서 진행이 될 것이다.
# 첫 번쨰는 팀을 무시한 년도별 타자들만 모아서 말 그대로 최적의 타순을 구하는 방식.
#이 방식은 온전히 이론적 최다득점의 타순을 구하기 위한 방식이다.

#그리고 두 번째는 팀까지 생각한 경우이다.
# 이 방법은 의도한 바와는 다른 결과가 나올 순 있을 지언정, 대신 현실적인 체감 타순에는 가장 가까운 것으로 보인다.

 팀/연도별 선수 수:
year  team
20    KIA      7
21    한화       7
22    키움       8
20    SK       8
24    키움       8
21    NC       8
22    한화       9
23    두산       9
      삼성       9
24    KT       9
21    LG       9
      롯데       9
20    두산       9
      롯데       9
24    롯데       9
20    삼성       9
21    KT       9
      KIA     10
20    KT      10
24    삼성      10
20    NC      10
22    LG      10
      KIA     10
21    삼성      10
24    LG      10
      NC      10
      두산      10
23    한화      10
      LG      10
      키움      11
      NC      11
      롯데      11
      SSG     11
22    SSG     11
23    KIA     11
22    두산      11
      NC      11
      KT      11
21    키움      11
      두산      11
      SSG     11
20    LG      11
23    KT      11
24    한화      11
22    삼성      12
      롯데      12
24    KIA     12
      SSG     12
20    키움      12
      한화      13
dtype: int64

 전체 선수 수 (조건 없음): 503


1

In [25]:
# PA 100 이상인 선수만 남긴 후 팀/연도별 개수 다시 확인한다. 적절한 개수, 충분한 객체수가 있는 지 확인하기 위해서다.
df_filtered = df_clean[df_clean['pa'] > 100].copy()
summary_check = df_filtered.groupby(['year', 'team']).size().sort_values()

print("팀/연도별 PA > 100 선수 수:")
print(summary_check)

# 이후 충분한 갯수가 있음을 확인하고 진행한다.


팀/연도별 PA > 100 선수 수:
year  team
20    KIA      7
21    한화       7
22    키움       8
20    SK       8
24    키움       8
21    NC       8
22    한화       9
23    두산       9
      삼성       9
24    KT       9
21    LG       9
      롯데       9
20    두산       9
      롯데       9
24    롯데       9
20    삼성       9
21    KT       9
      KIA     10
20    KT      10
24    삼성      10
20    NC      10
22    LG      10
      KIA     10
21    삼성      10
24    LG      10
      NC      10
      두산      10
23    한화      10
      LG      10
      키움      11
      NC      11
      롯데      11
      SSG     11
22    SSG     11
23    KIA     11
22    두산      11
      NC      11
      KT      11
21    키움      11
      두산      11
      SSG     11
20    LG      11
23    KT      11
24    한화      11
22    삼성      12
      롯데      12
24    KIA     12
      SSG     12
20    키움      12
      한화      13
dtype: int64


In [26]:
# PA 100 이상 필터링 (전처리된 df_clean 사용)
df_filtered = df_clean[df_clean['pa'] > 100].copy()

# 팀/연도별 상위 9명 추출 (Pandas 향후 버전 대비)
top_batters_by_team = (
    df_filtered.sort_values(by='pa', ascending=False)
    .groupby(['year', 'team'], group_keys=False)
    .head(9)
    .reset_index(drop=True)
)

# obp, slg 타입 안전 변환
top_batters_by_team['obp'] = pd.to_numeric(top_batters_by_team['obp'], errors='coerce').fillna(0.0)
top_batters_by_team['slg'] = pd.to_numeric(top_batters_by_team['slg'], errors='coerce').fillna(0.0)

# 전체 데이터 크기 확인
print(" top_batters_by_team 전체 행 수:", len(top_batters_by_team))
print(" 컬럼 목록:", top_batters_by_team.columns.tolist())

# 연도별, 팀별 몇 명씩 있는지 확인 (9명씩 있어야 함)
print("\n 팀/연도별 선수 수:")
print(top_batters_by_team.groupby(['year', 'team']).size())

# 샘플 상위 10개 확인
print("\n상위 10개 샘플:")
display(top_batters_by_team.head(10))

 top_batters_by_team 전체 행 수: 442
 컬럼 목록: ['year', 'team', 'name', 'pa', 'ops', 'avg', 'obp', 'slg', 'rbi', 'sb', 'hr']

 팀/연도별 선수 수:
year  team
20    KIA     7
      KT      9
      LG      9
      NC      9
      SK      8
      두산      9
      롯데      9
      삼성      9
      키움      9
      한화      9
21    KIA     9
      KT      9
      LG      9
      NC      8
      SSG     9
      두산      9
      롯데      9
      삼성      9
      키움      9
      한화      7
22    KIA     9
      KT      9
      LG      9
      NC      9
      SSG     9
      두산      9
      롯데      9
      삼성      9
      키움      8
      한화      9
23    KIA     9
      KT      9
      LG      9
      NC      9
      SSG     9
      두산      9
      롯데      9
      삼성      9
      키움      9
      한화      9
24    KIA     9
      KT      9
      LG      9
      NC      9
      SSG     9
      두산      9
      롯데      9
      삼성      9
      키움      8
      한화      9
dtype: int64

상위 10개 샘플:


,year,team,name,pa,ops,avg,obp,slg,rbi,sb,hr
0,24,KT,로하스,670,0.989,0.329,0.421,0.568,112,2,32
1,20,두산,페르난데스,668,0.901,0.340,0.404,0.497,105,0,21
2,21,KIA,최원준,668,0.742,0.295,0.370,0.372,44,40,4
3,21,LG,홍창기,651,0.864,0.328,0.456,0.408,52,23,4
4,22,KIA,나성범,649,0.910,0.320,0.402,0.508,97,6,21
5,22,한화,터크먼,648,0.796,0.289,0.366,0.430,43,19,12
6,23,LG,홍창기,643,0.856,0.332,0.444,0.412,65,23,1
7,22,SSG,최지훈,640,0.789,0.304,0.362,0.427,61,31,10
8,24,LG,홍창기,637,0.857,0.336,0.447,0.410,73,10,5
9,22,LG,박해민,636,0.715,0.289,0.347,0.368,49,24,3


In [27]:
missing_check = df_all[['year', 'team', 'name', 'pa', 'obp', 'slg']].isnull().sum()
print("결측치 개수:")
print(missing_check)

missing_rows = df_all[df_all[['pa', 'obp', 'slg']].isnull().any(axis=1)]
print("결측치 포함된 샘플:")
print(missing_rows[['year', 'team', 'name', 'pa', 'obp', 'slg']].head(10))

결측치 개수:
year    0
team    0
name    0
pa      0
obp     0
slg     0
dtype: int64
결측치 포함된 샘플:
Empty DataFrame
Columns: [year, team, name, pa, obp, slg]
Index: []


In [28]:
import random

# 전체 연도/팀 목록 추출
years = top_batters_by_team['year'].unique()
teams = top_batters_by_team['team'].unique()

# 초기 세대 생성 함수
def generate_chromosome(batter_names):
    return random.sample(batter_names, len(batter_names))
#여기서 세대 수를 100으로 설정한 것은 연산의 한계점을 고려한 결정이다. 더 높은 세대 수를 설정할 수록 더욱 다양한 결과를 얻을 수 있을 것으로 기대된다.


def create_initial_population(batter_names, size=100):
    return [generate_chromosome(batter_names) for _ in range(size)]


In [29]:
import pandas as pd



# 컬럼명 소문자 통일
df_all.columns = [col.strip().lower() for col in df_all.columns]

# 필요한 컬럼 확인
expected_cols = ['year', 'team', 'name', 'pa', 'ops', 'avg', 'obp', 'slg', 'rbi', 'sb', 'hr']
missing_cols = [col for col in expected_cols if col not in df_all.columns]

result = {}
result['누락된 컬럼'] = missing_cols

# 결측 없이 데이터 필터링
if not missing_cols:
    df_all = df_all[expected_cols].dropna(subset=['year', 'team', 'name', 'pa', 'obp', 'slg'])

    # pa > 100 필터링
    df_filtered = df_all[df_all['pa'] > 100].copy()
    df_filtered['obp'] = pd.to_numeric(df_filtered['obp'], errors='coerce')
    df_filtered['slg'] = pd.to_numeric(df_filtered['slg'], errors='coerce')

    # 팀별 타자 수
    team_counts = (
        df_filtered.groupby(['year', 'team'])['name']
        .nunique()
        .reset_index(name='num_players')
    )

    insufficient_teams = team_counts[team_counts['num_players'] < 9]
    sufficient_teams = team_counts[team_counts['num_players'] >= 9]

    # 결측 선수 수
    obp_slg_na = df_filtered[df_filtered['obp'].isna() | df_filtered['slg'].isna()]

    result_summary = {
        "전체 팀 수": len(team_counts),
        "9명 미만 팀 수": len(insufficient_teams),
        "9명 이상 팀 수": len(sufficient_teams),
        "OBP/SLG 결측 선수 수": len(obp_slg_na)
    }

    team_counts = (
    top_batters_by_team.groupby(['year', 'team'])['name']
    .nunique()
    .reset_index(name='num_players')
)

    valid_teams = team_counts[team_counts['num_players'] >= 9]
print("유효한 팀 개수:", len(valid_teams))

# 유효한 팀-연도 쌍만 필터링
top_batters_by_team = top_batters_by_team.merge(valid_teams[['year', 'team']], on=['year', 'team'], how='inner')


print(team_counts.sort_values('num_players'))  # 몇 명이 포함되었는지 확인


result_summary


유효한 팀 개수: 44
    year team  num_players
0     20  KIA            7
19    21   한화            7
28    22   키움            8
4     20   SK            8
48    24   키움            8
13    21   NC            8
29    22   한화            9
30    23  KIA            9
31    23   KT            9
32    23   LG            9
33    23   NC            9
34    23  SSG            9
35    23   두산            9
36    23   롯데            9
37    23   삼성            9
39    23   한화            9
40    24  KIA            9
41    24   KT            9
42    24   LG            9
43    24   NC            9
44    24  SSG            9
45    24   두산            9
46    24   롯데            9
47    24   삼성            9
38    23   키움            9
27    22   삼성            9
24    22  SSG            9
25    22   두산            9
1     20   KT            9
2     20   LG            9
3     20   NC            9
5     20   두산            9
6     20   롯데            9
7     20   삼성            9
8     20   키움            9
9     20   한화  

{'전체 팀 수': 50, '9명 미만 팀 수': 6, '9명 이상 팀 수': 44, 'OBP/SLG 결측 선수 수': 0}

In [30]:
import pandas as pd

# 컬럼 소문자 변환
df_all.columns = [col.strip().lower() for col in df_all.columns]

# 필요한 컬럼 체크
expected_cols = ['year', 'team', 'name', 'pa', 'ops', 'avg', 'obp', 'slg', 'rbi', 'sb', 'hr']
missing_cols = [col for col in expected_cols if col not in df_all.columns]

# 결과 저장용 딕셔너리
result = {}
result['누락된 컬럼'] = missing_cols

if not missing_cols:
    df_all = df_all[expected_cols].dropna(subset=['year', 'team', 'name', 'pa', 'obp', 'slg'])

    df_filtered = df_all[df_all['pa'] > 100].copy()
    df_filtered['obp'] = pd.to_numeric(df_filtered['obp'], errors='coerce')
    df_filtered['slg'] = pd.to_numeric(df_filtered['slg'], errors='coerce')

    # 팀-연도별 선수 수 계산
    team_counts = (
        df_filtered.groupby(['year', 'team'])['name']
        .nunique()
        .reset_index(name='num_players')
    )

    insufficient_teams = team_counts[team_counts['num_players'] < 9]
    sufficient_teams = team_counts[team_counts['num_players'] >= 9]

    result['선수 수 부족한 팀 목록'] = insufficient_teams
    result['선수 수 충분한 팀 목록'] = sufficient_teams

    # 결측 OBP/SLG 확인
    obp_slg_na = df_filtered[df_filtered['obp'].isna() | df_filtered['slg'].isna()]
    result['OBP/SLG 결측 선수 수'] = len(obp_slg_na)

    #  출력 (Colab 호환)
    print(" 선수 수 부족한 팀 목록:")
    display(insufficient_teams)

    print(" 선수 수 충분한 팀 목록:")
    display(sufficient_teams)

    print(" OBP/SLG 결측 선수 수:", len(obp_slg_na))

# 요약 통계
result_summary = {
    "전체 팀 수": len(team_counts),
    "9명 미만 팀 수": len(insufficient_teams),
    "9명 이상 팀 수": len(sufficient_teams),
    "OBP/SLG 결측 선수 수": len(obp_slg_na)
}

# 출력
print(" 요약 통계:")
for k, v in result_summary.items():
    print(f"{k}: {v}")


 선수 수 부족한 팀 목록:


,year,team,num_players
0,20,KIA,7
4,20,SK,8
13,21,NC,8
19,21,한화,7
28,22,키움,8
48,24,키움,8


 선수 수 충분한 팀 목록:


,year,team,num_players
1,20,KT,10
2,20,LG,11
3,20,NC,10
5,20,두산,9
6,20,롯데,9
7,20,삼성,9
8,20,키움,12
9,20,한화,13
10,21,KIA,10
11,21,KT,9


 OBP/SLG 결측 선수 수: 0
 요약 통계:
전체 팀 수: 50
9명 미만 팀 수: 6
9명 이상 팀 수: 44
OBP/SLG 결측 선수 수: 0


In [15]:
import os

# 드라이브 경로 설정
#folder_path = "/content/drive/MyDrive/GA_result_logs"
folder_path = os.path.join(os.getcwd(), "GA_result_logs")

# 폴더 존재 여부 확인
if os.path.exists(folder_path):
    print(f" 폴더 존재: {folder_path}")
    files = os.listdir(folder_path)

    if files:
        print(" 폴더 내 파일 목록:")
        for f in files:
            print(" -", f)
    else:
        print(" 폴더는 비어 있습니다.")
else:
    print(" 폴더가 존재하지 않습니다. 경로를 확인하세요.")


 폴더가 존재하지 않습니다. 경로를 확인하세요.


In [ ]:
# 유전 알고리즘 기반 타순 최적화
# 코드 구성 요소:
# - 진루 처리
# - 적합도 계산
# - 엘리티즘 포함한 세대 교체
# - 토너먼트 선택 방식 적용
# - JSON 저장 및 중간 기록 저장
# - 병렬 처리로 속도 향상

import os
import json
import random
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from datetime import datetime

# ------------------ 저장 경로 설정 ------------------
#SAVE_PATH = "/content/drive/MyDrive/GA_result_logs"
SAVE_PATH = os.path.join(os.getcwd(), "GA_result_logs")

os.makedirs(SAVE_PATH, exist_ok=True)



# ------------------ 유틸: 진루 처리 ------------------
def advance_runners(bases, hit_type):
    runs = 0
    if hit_type == 4:
        runs += sum(bases) + 1
        return [0, 0, 0], runs
    for i in reversed(range(3)):
        if bases[i] == 1:
            if i + hit_type >= 3:
                runs += 1
            else:
                bases[i + hit_type] = 1
            bases[i] = 0
    bases[hit_type - 1] = 1
    return bases, runs

# ------------------ 적합도 계산 ------------------
def calculate_fitness(lineup, batter_stats, num_simulations=200):
    total_runs = 0
    for _ in range(num_simulations):
        bases = [0, 0, 0]
        outs = 0
        runs = 0
        batter_index = 0
        for _ in range(9):  # 기존 방식 유지
            outs = 0
            bases = [0, 0, 0]
            while outs < 3:
                current_batter = lineup[batter_index % len(lineup)]
                batter_data = batter_stats[batter_stats['name'] == current_batter]

                if batter_data.empty:
                    outs += 1
                    batter_index += 1
                    continue

                try:
                    obp = float(batter_data['obp'].values[0])
                    slg = float(batter_data['slg'].values[0])
                except (IndexError, ValueError, TypeError):
                    outs += 1
                    batter_index += 1
                    continue

                if pd.isna(obp) or obp == 0 or slg == 0:
                    outs += 1
                    batter_index += 1
                    continue


                if random.random() < obp:
                    hit_type = random.choices([1,2,3,4], weights=[slg*0.6, slg*0.25, slg*0.1, slg*0.05])[0]
                    bases, new_runs = advance_runners(bases, hit_type)
                    runs += new_runs
                else:
                    outs += 1
                batter_index += 1

        total_runs += runs

    return total_runs / num_simulations

# ------------------ GA 기본 ------------------
def generate_chromosome(names):
    return random.sample(names, len(names))

def create_initial_population(names, size=100):
    return [generate_chromosome(names) for _ in range(size)]

def tournament_selection(population, fitness_scores, k=5):
    selected = random.sample(list(zip(population, fitness_scores)), k)
    return max(selected, key=lambda x: x[1])[0]

def crossover(p1, p2):
    a, b = sorted(random.sample(range(len(p1)), 2))
    child = [None]*len(p1)
    child[a:b+1] = p1[a:b+1]
    fill = [x for x in p2 if x not in child]
    j = 0
    for i in range(len(p1)):
        if child[i] is None:
            child[i] = fill[j]
            j += 1
    return child

def mutate(chrom, rate=0.1):
    if random.random() < rate:
        a, b = random.sample(range(len(chrom)), 2)
        chrom[a], chrom[b] = chrom[b], chrom[a]
    return chrom

# ------------------ 유닛 테스트 ------------------
def test_advance_runners():
    assert advance_runners([1, 1, 1], 4) == ([0, 0, 0], 4)
    assert advance_runners([0, 1, 0], 1) == ([1, 0, 1], 0)
    print("진루 함수 테스트 통과")

test_advance_runners()

# ------------------ 메인 루프 ------------------
results = []
all_top_lineups = []
population_size = 50
generations = 30
mutation_rate = 0.02

for (year, team), group in top_batters_by_team.groupby(['year', 'team']):
    if len(group) != 9:
        print(f"{year} {team} 스킵 (타자 수 부족)")
        continue

    key = f"{year}_{team}"
    file_path = os.path.join(SAVE_PATH, f"{key}.json")
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    # 기존 결과 불러오기 (있다면)
    generations_data = []
    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as f:
            generations_data = json.load(f)
        if len(generations_data) >= generations:
            print(f"{key} - 이미 {len(generations_data)}세대 완료됨, 스킵")
            continue
        else:
            print(f"{key} - {len(generations_data)}세대까지 완료됨, 이어서 수행")

    # 초기화
    batters = group.reset_index(drop=True)
    names = batters['name'].tolist()
    population = create_initial_population(names, population_size)

    # 적합도 계산
    from multiprocessing import Pool, cpu_count
    with Pool(cpu_count()) as pool:
        fitness_scores = pool.starmap(calculate_fitness, [(ind, batters) for ind in population])

    # 이미 진행된 세대 수만큼 건너뛰기
    for g in range(len(generations_data), generations):
        best_index = np.argmax(fitness_scores)
        best = population[best_index]
        best_score = fitness_scores[best_index]

        print(f"{key} - Gen {g+1}: 기대 득점 = {round(best_score, 3)}")

        lineup_stats = batters.set_index('name').loc[best].reset_index().to_dict(orient='records')
        generations_data.append({
            'generation': g + 1,
            'expected_score': round(best_score, 3),
            'lineup': best,
            'player_stats': lineup_stats
        })

        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(generations_data, f, ensure_ascii=False, indent=2)

        # 다음 세대 준비
        new_population = [best]
        while len(new_population) < population_size:
            p1 = tournament_selection(population, fitness_scores)
            p2 = tournament_selection(population, fitness_scores)
            child = mutate(crossover(p1, p2), mutation_rate)
            new_population.append(child)

        population = new_population
        with Pool(cpu_count()) as pool:
            fitness_scores = pool.starmap(calculate_fitness, [(ind, batters) for ind in population])

    results.append({
        'year': year,
        'team': team,
        'lineup': best,
        'expected_runs': round(best_score, 3)
    })



# 저장할 파일 경로 정의
excel_output_path = os.path.join(SAVE_PATH, "최적_라인업_결과.xlsx")

# 저장 실행
pd.DataFrame(results).to_excel(excel_output_path, index=False)


진루 함수 테스트 통과


In [31]:
# 유전 알고리즘 파라미터 설정

# 개체군 크기: 100
# 9명의 타순 조합은 총 9! = 362,880가지의 경우가 있으므로,
# 초기 탐색 범위 확보를 위해 개체군은 최소 80~100 이상이 바람직함
# 개체군이 클수록 다양한 조합을 초기에 탐색할 수 있어 국지 최적에 빠질 위험이 줄어듦
# 하지만 너무 클 경우 연산 속도와 리소스 소모가 증가하므로, 성능 균형을 고려해 100으로 설정


 # 탐색 공간 확대
# 세대 수: 100
# 적합도 함수가 시뮬레이션 기반으로 계산량이 크기 때문에, 너무 많은 세대를 설정하면 비효율적
# 일반적으로 30~50 세대 내에서 최적화가 충분히 수렴됨
# 특히 OPS 기반으로 초기 필터링된 타자들이라면, 세대 수를 40 정도로 두는 것이 현실적 효율성 확보에 적합


# 최적 밸런스 유지


In [32]:
import os
import json
import pandas as pd

#SAVE_PATH = "/content/drive/MyDrive/GA_result_logs"
SAVE_PATH = os.path.join(os.getcwd(), "GA_result_logs")

all_top_lineups = []

# 모든 JSON 파일 순회
for file_name in os.listdir(SAVE_PATH):
    if file_name.endswith(".json"):
        year_team = file_name.replace(".json", "")  # 예: 2023_KIA
        year, team = year_team.split("_")

        with open(os.path.join(SAVE_PATH, file_name), 'r', encoding='utf-8') as f:
            generations_data = json.load(f)

            for gen in generations_data:
                all_top_lineups.append({
                    "generation": gen["generation"],
                    "rank": 1,  # 상위 1개만 저장된 구조니까
                    "score": round(gen["expected_score"], 3),
                    "lineup": gen["lineup"],
                    "year": int(year),
                    "team": team
                })

# 5. 데이터프레임 변환 및 상위 5개만 보기
df_lineups = pd.DataFrame(results)

# 상위 점수순 정렬
df_lineups = df_lineups.sort_values(by=["generation", "score"], ascending=[True, False])

import ace_tools as tools; tools.display_dataframe_to_user(name="최적 라인업 요약", dataframe=df_lineups.head(15))

NameError: name 'results' is not defined

In [33]:
print(df_lineups.columns.tolist())

NameError: name 'df_lineups' is not defined